# Run Molecular Dynamics Trajectories with Psi4
We want to see how closely our ML models adhere to trajectories produced without surrogates

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from fff.sampling.md import MolecularDynamics
from fff.simulation.utils import read_from_string
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.calculators.psi4 import Psi4
from ase.db import connect
from ase.io import write
from ttm.ase import TTMCalculator
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings

Configuration

In [ ]:
out_dir = Path('trajectories')

## Load in Example Structures
Use the same ones from our nwchem-evaluation

## Run Molecular Dynamics with DFT
Run a 1000 timesteps and save an entry every 10

In [ ]:
psi4 = Psi4(method='pbe0-d3', basis='aug-cc-pvdz', num_threads=12)

Test with diff

In [ ]:
out_dir.mkdir(exist_ok=True)
for name, calc in [('dft', psi4)]:
    for temp in [100, 300, 900]:
        with connect('methane-added.db') as db:
            for row in tqdm(db.select('', limit=4)):
                out_path = out_dir / f'{name}_n={(row.natoms - 2) // 3}_T={temp}K-{row.filename}.db'
                if out_path.is_file():
                    continue

                # Read the structure and initialize it at 100K
                atoms = row.toatoms()
                MaxwellBoltzmannDistribution(atoms, temperature_K=temp * 2,
                                             rng=np.random.RandomState(1))

                # Run it for a few timesteps
                with warnings.catch_warnings():
                    warnings.simplefilter('ignore')
                    _, traj = MolecularDynamics().run_sampling(atoms, 32, calc, timestep=0.1, log_interval=2)

                # Save it to disk
                write(str(out_path), traj)